In [1]:
# run this first cell
!pip install --upgrade ultralytics opencv-python pillow tqdm matplotlib pandas


   ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
   ---------------------------------------- 1.1/1.1 MB 7.5 MB/s  0:00:00
   ---------------------------------------- 0.0/39.0 MB ? eta -:--:--
   -- ------------------------------------- 2.4/39.0 MB 12.2 MB/s eta 0:00:04
   ------ --------------------------------- 6.6/39.0 MB 16.8 MB/s eta 0:00:02
   -------------- ------------------------- 14.2/39.0 MB 23.4 MB/s eta 0:00:02
   ------------------------- -------------- 24.6/39.0 MB 30.0 MB/s eta 0:00:01
   -------------------------------------- - 37.2/39.0 MB 36.4 MB/s eta 0:00:01
   ---------------------------------------- 39.0/39.0 MB 35.4 MB/s  0:00:01
   ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
   ---------------------------------------- 12.9/12.9 MB 62.3 MB/s  0:00:00
   ---------------------------------------- 0.0/7.0 MB ? eta -:--:--
   ---------------------------------------- 7.0/7.0 MB 53.9 MB/s  0:00:00
   ---------------------------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit 1.50.0 requires pillow<12,>=7.1.0, but you have pillow 12.0.0 which is incompatible.


In [2]:
from pathlib import Path

BASE = Path(r"C:\Final project 2\backend\datasets\FracAtlas")
IMAGES = BASE / "images"
LABELS_SRC = BASE / "Annotations" / "YOLO"   # YOLO labels exist here
print("Images folder:", IMAGES)
print("Labels folder:", LABELS_SRC)


Images folder: C:\Final project 2\backend\datasets\FracAtlas\images
Labels folder: C:\Final project 2\backend\datasets\FracAtlas\Annotations\YOLO


In [3]:
from PIL import Image
import shutil

# target clean dataset folders
for split in ["train", "val", "test"]:
    (BASE / f"images/{split}").mkdir(parents=True, exist_ok=True)
    (BASE / f"labels/{split}").mkdir(parents=True, exist_ok=True)

all_imgs = list(IMAGES.glob("*.jpg")) + list(IMAGES.glob("*.png"))
valid_pairs = []

for img in all_imgs:
    lbl = LABELS_SRC / (img.stem + ".txt")
    if lbl.exists():
        try:
            with Image.open(img) as im:
                im.verify()
            valid_pairs.append((img, lbl))
        except Exception:
            continue

print(f"✅ Found {len(valid_pairs)} clean image/label pairs.")


✅ Found 0 clean image/label pairs.


In [4]:
from pathlib import Path
import shutil, random
from PIL import Image

BASE = Path(r"C:\Final project 2\backend\datasets\FracAtlas")
FRACTURED_DIR = BASE / "images" / "Fractured"
NON_FRACTURED_DIR = BASE / "images" / "Non_fractured"
YOLO_LABELS = BASE / "Annotations" / "YOLO"

print(FRACTURED_DIR.exists(), NON_FRACTURED_DIR.exists(), YOLO_LABELS.exists())


True True True


In [5]:
img_exts = {".jpg", ".jpeg", ".png", ".bmp"}

fractured_imgs = [p for p in FRACTURED_DIR.rglob("*") if p.suffix.lower() in img_exts]
nonfractured_imgs = [p for p in NON_FRACTURED_DIR.rglob("*") if p.suffix.lower() in img_exts]

print("fractured imgs:", len(fractured_imgs))
print("non-fractured imgs:", len(nonfractured_imgs))

all_imgs = fractured_imgs + nonfractured_imgs
print("total imgs:", len(all_imgs))


fractured imgs: 717
non-fractured imgs: 3366
total imgs: 4083


In [6]:
for split in ["train", "val", "test"]:
    (BASE / f"images/{split}").mkdir(parents=True, exist_ok=True)
    (BASE / f"labels/{split}").mkdir(parents=True, exist_ok=True)


In [7]:
items = []
for img in all_imgs:
    lbl = YOLO_LABELS / f"{img.stem}.txt"
    has_label = lbl.exists()
    # also skip corrupted images
    try:
        with Image.open(img) as im:
            im.verify()
        items.append((img, lbl, has_label))
    except Exception:
        pass

print("usable images:", len(items))
print("with labels:", sum(1 for _,_,h in items if h))
print("without labels (non-fractured):", sum(1 for _,_,h in items if not h))


usable images: 4083
with labels: 4083
without labels (non-fractured): 0


In [8]:
random.shuffle(items)
n = len(items)
train_n = int(0.8 * n)
val_n   = int(0.1 * n)

splits = {
    "train": items[:train_n],
    "val":   items[train_n:train_n+val_n],
    "test":  items[train_n+val_n:]
}
for k,v in splits.items():
    print(k, len(v))


train 3266
val 408
test 409


In [9]:
for split, rows in splits.items():
    for img, lbl, has_label in rows:
        # copy image
        dst_img = BASE / f"images/{split}" / img.name
        shutil.copy2(img, dst_img)

        # label name
        dst_lbl = BASE / f"labels/{split}" / f"{img.stem}.txt"
        if has_label:
            shutil.copy2(lbl, dst_lbl)
        else:
            # create an empty label file so YOLO won't complain
            dst_lbl.write_text("", encoding="utf-8")

print("✅ copied images and labels (empty for non-fractured)")


✅ copied images and labels (empty for non-fractured)


In [7]:
yaml_text = """
path: C:/Final project 2/backend/datasets/FracAtlas
train: images/train
val: images/val
test: images/test

names:
  0: fracture
"""
yaml_file = BASE / "fracatlas.yaml"
yaml_file.write_text(yaml_text, encoding="utf-8")
print("✅ wrote", yaml_file)


✅ wrote C:\Final project 2\backend\datasets\FracAtlas\fracatlas.yaml


In [1]:
!pip install albumentations==1.4.6 opencv-python pillow tqdm


   ---------------------------------------- 0.0/38.9 MB ? eta -:--:--
   - -------------------------------------- 1.0/38.9 MB 5.6 MB/s eta 0:00:07
   -- ------------------------------------- 2.6/38.9 MB 6.9 MB/s eta 0:00:06
   ---- ----------------------------------- 4.5/38.9 MB 7.4 MB/s eta 0:00:05
   ------- -------------------------------- 7.6/38.9 MB 9.8 MB/s eta 0:00:04
   ------------- -------------------------- 13.4/38.9 MB 13.5 MB/s eta 0:00:02
   ---------------------- ----------------- 22.3/38.9 MB 18.5 MB/s eta 0:00:01
   ------------------------------- -------- 30.4/38.9 MB 21.4 MB/s eta 0:00:01
   ---------------------------------------  38.8/38.9 MB 23.9 MB/s eta 0:00:01
   ---------------------------------------- 38.9/38.9 MB 23.3 MB/s  0:00:01
   ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
   ------------------------- -------------- 8.1/12.8 MB 38.7 MB/s eta 0:00:01
   ---------------------------------------- 12.8/12.8 MB 38.3 MB/s  0:00:00
   ---

In [2]:
from pathlib import Path
from tqdm import tqdm

BASE = Path(r"C:\Final project 2\backend\datasets\FracAtlas")
TRAIN_IMG = BASE / "images/train"
TRAIN_LBL = BASE / "labels/train"

fractured, nonfractured = [], []

for img in TRAIN_IMG.glob("*.*"):
    lbl = TRAIN_LBL / f"{img.stem}.txt"
    if lbl.exists() and lbl.stat().st_size > 0:
        fractured.append(img)
    else:
        nonfractured.append(img)

print(f"Fractured: {len(fractured)}, Non-fractured: {len(nonfractured)}")


Fractured: 698, Non-fractured: 3229


In [3]:
import cv2
import numpy as np
from albumentations import (
    Compose,
    RandomBrightnessContrast,
    GaussNoise,
    GaussianBlur,
    RGBShift,
    CLAHE
)

AUG = Compose([
    RandomBrightnessContrast(p=0.7),
    GaussNoise(var_limit=(5.0, 30.0), p=0.4),
    GaussianBlur(blur_limit=3, p=0.3),
    RGBShift(r_shift_limit=10, g_shift_limit=10, b_shift_limit=10, p=0.3),
    CLAHE(p=0.3),
])


c:\Users\aadit\anaconda3\envs\fxdetector\lib\site-packages\albumentations\augmentations\blur\transforms.py:240: UserWarning: blur_limit and sigma_limit minimum value can not be both equal to 0. blur_limit minimum value changed to 3.
  warnings.warn(


In [4]:
from PIL import Image
from tqdm import tqdm

target = len(nonfractured)          # 3,229
current = len(fractured)            # 698
to_make = target - current          # 2,531

print("Need to create", to_make, "augmented fractured images")

AUG_OUT = TRAIN_IMG   # we will just drop them into images/train
made = 0

# loop over existing fractured images again and again until we reach target
while made < to_make:
    for img_path in fractured:
        if made >= to_make:
            break

        # read image
        img = cv2.imread(str(img_path))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        aug = AUG(image=img)["image"]

        out_name = f"{img_path.stem}_aug_{made}.jpg"
        out_path = AUG_OUT / out_name

        # save augmented img
        Image.fromarray(aug).save(out_path, quality=95)

        # copy same label
        lbl_src = TRAIN_LBL / f"{img_path.stem}.txt"
        lbl_dst = TRAIN_LBL / f"{out_path.stem}.txt"
        if lbl_src.exists():
            lbl_dst.write_text(lbl_src.read_text(encoding="utf-8"), encoding="utf-8")

        made += 1

print("✅ done. created", made, "augmented fractured images")


Need to create 2531 augmented fractured images
✅ done. created 2531 augmented fractured images


In [5]:
fractured2 = []
nonfractured2 = []

for img in TRAIN_IMG.glob("*.*"):
    lbl = TRAIN_LBL / f"{img.stem}.txt"
    if lbl.exists() and lbl.stat().st_size > 0:
        fractured2.append(img)
    else:
        nonfractured2.append(img)

print("After augmentation:")
print("Fractured:", len(fractured2))
print("Non-fractured:", len(nonfractured2))


After augmentation:
Fractured: 3229
Non-fractured: 3229


In [9]:
from ultralytics import YOLO

model = YOLO("yolov8s.pt")

model.train(
    data=r"C:\Final project 2\backend\datasets\FracAtlas\fracatlas.yaml",
    epochs=100,
    imgsz=640,
    batch=8,
    project=r"C:\Final project 2\backend\runs",
    name="FracAtlas-balanced",
)


Ultralytics 8.3.225  Python-3.10.18 torch-2.8.0+cu129 CUDA:0 (NVIDIA GeForce RTX 5070 Laptop GPU, 8151MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:\Final project 2\backend\datasets\FracAtlas\fracatlas.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=FracAtlas-balanced2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto

c:\Users\aadit\anaconda3\envs\fxdetector\lib\site-packages\albumentations\core\validation.py:34: UserWarning: Argument 'quality_range' is not valid and will be ignored.
  warn(


val: Fast image access  (ping: 0.00.0 ms, read: 224.698.6 MB/s, size: 16.1 KB)
val: Scanning C:\Final project 2\backend\datasets\FracAtlas\labels\val.cache... 762 images, 629 backgrounds, 12 corrupt: 100% ━━━━━━━━━━━━ 774/774 774.4Kit/s 0.0s
val: C:\Final project 2\backend\datasets\FracAtlas\images\val\IMG0004029.jpg: ignoring corrupt image/label: image file is truncated (22 bytes not processed)
val: C:\Final project 2\backend\datasets\FracAtlas\images\val\IMG0004036.jpg: ignoring corrupt image/label: image file is truncated (14 bytes not processed)
val: C:\Final project 2\backend\datasets\FracAtlas\images\val\IMG0004070.jpg: ignoring corrupt image/label: image file is truncated (41 bytes not processed)
val: C:\Final project 2\backend\datasets\FracAtlas\images\val\IMG0004100.jpg: ignoring corrupt image/label: image file is truncated (15 bytes not processed)
val: C:\Final project 2\backend\datasets\FracAtlas\images\val\IMG0004121.jpg: ignoring corrupt image/label: image file is truncate

c:\Users\aadit\anaconda3\envs\fxdetector\lib\site-packages\albumentations\core\validation.py:34: UserWarning: Argument 'quality_range' is not valid and will be ignored.
  warn(



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
     91/100      2.57G     0.6624     0.3813     0.8478          3        640: 100% ━━━━━━━━━━━━ 801/801 10.1it/s 1:19<0.1ss
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 48/48 14.3it/s 3.3s0.1s
                   all        762        170      0.986      0.976      0.976      0.848

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
     92/100      2.57G     0.6537     0.3736     0.8451          2        640: 100% ━━━━━━━━━━━━ 801/801 10.2it/s 1:19<0.2ss
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 48/48 14.3it/s 3.4s0.1s
                   all        762        170      0.988      0.974      0.976      0.847

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
     93/100      2.57G     0.6386     0.3734     0.8438          5     

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000023C6A319810>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0480

In [8]:
from ultralytics import YOLO

# install once in another cell:
# !pip install ultralytics

model = YOLO("yolov8s.pt")

model.train(
    data=str(yaml_file),
    epochs=100,
    imgsz=640,
    batch=8,
    project=r"C:\Final project 2\backend\runs",
    name="FracAtlas-balanced",
)


Ultralytics 8.3.225  Python-3.10.18 torch-2.8.0+cu129 CUDA:0 (NVIDIA GeForce RTX 5070 Laptop GPU, 8151MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:\Final project 2\backend\datasets\FracAtlas\fracatlas.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=FracAtlas-balanced, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto,

c:\Users\aadit\anaconda3\envs\fxdetector\lib\site-packages\albumentations\core\validation.py:34: UserWarning: Argument 'quality_range' is not valid and will be ignored.
  warn(


val: Fast image access  (ping: 0.00.0 ms, read: 210.177.3 MB/s, size: 17.9 KB)
val: Scanning C:\Final project 2\backend\datasets\FracAtlas\labels\val.cache... 762 images, 629 backgrounds, 12 corrupt: 100% ━━━━━━━━━━━━ 774/774 387.2Kit/s 0.0s
val: C:\Final project 2\backend\datasets\FracAtlas\images\val\IMG0004029.jpg: ignoring corrupt image/label: image file is truncated (22 bytes not processed)
val: C:\Final project 2\backend\datasets\FracAtlas\images\val\IMG0004036.jpg: ignoring corrupt image/label: image file is truncated (14 bytes not processed)
val: C:\Final project 2\backend\datasets\FracAtlas\images\val\IMG0004070.jpg: ignoring corrupt image/label: image file is truncated (41 bytes not processed)
val: C:\Final project 2\backend\datasets\FracAtlas\images\val\IMG0004100.jpg: ignoring corrupt image/label: image file is truncated (15 bytes not processed)
val: C:\Final project 2\backend\datasets\FracAtlas\images\val\IMG0004121.jpg: ignoring corrupt image/label: image file is truncate

KeyboardInterrupt: 

In [10]:
metrics = model.val()
print(f"mAP50: {metrics.box.map50:.3f}")
print(f"mAP50-95: {metrics.box.map:.3f}")
print(f"Precision: {metrics.box.p.mean():.3f}")
print(f"Recall: {metrics.box.r.mean():.3f}")


Ultralytics 8.3.225  Python-3.10.18 torch-2.8.0+cu129 CUDA:0 (NVIDIA GeForce RTX 5070 Laptop GPU, 8151MiB)
Model summary (fused): 72 layers, 11,125,971 parameters, 0 gradients, 28.4 GFLOPs
val: Fast image access  (ping: 0.10.1 ms, read: 274.135.3 MB/s, size: 16.9 KB)
val: Scanning C:\Final project 2\backend\datasets\FracAtlas\labels\val.cache... 762 images, 629 backgrounds, 12 corrupt: 100% ━━━━━━━━━━━━ 774/774 772.2Kit/s 0.0s
val: C:\Final project 2\backend\datasets\FracAtlas\images\val\IMG0004029.jpg: ignoring corrupt image/label: image file is truncated (22 bytes not processed)
val: C:\Final project 2\backend\datasets\FracAtlas\images\val\IMG0004036.jpg: ignoring corrupt image/label: image file is truncated (14 bytes not processed)
val: C:\Final project 2\backend\datasets\FracAtlas\images\val\IMG0004070.jpg: ignoring corrupt image/label: image file is truncated (41 bytes not processed)
val: C:\Final project 2\backend\datasets\FracAtlas\images\val\IMG0004100.jpg: ignoring corrupt ima

In [11]:
!pip install ultralytics scikit-learn opencv-python pillow tqdm


In [2]:
from pathlib import Path
from ultralytics import YOLO

# 1. paths (your structure)
BASE = Path(r"C:\Final project 2\backend\datasets")
SPLITS = {
    "train": BASE / "train" / "images",
    "val":   BASE / "valid" / "images",
    "test":  BASE / "test" / "images",
}

# labels are in parallel folders
LABEL_BASE = {
    "train": BASE / "train" / "labels",
    "val":   BASE / "valid" / "labels",
    "test":  BASE / "test" / "labels",
}

# 2. load your fracture model (1-class)
model = YOLO(r"C:\Final project 2\backend\runs\FracAtlas-balanced2\weights\best.pt")

# in your data.yaml:
# nc: 10
# names: ['Comminuted', 'Greenstick', 'Healthy', 'Linear', 'Oblique Displaced', 'Oblique', 'Segmental', 'Spiral', 'Transverse Displaced', 'Transverse']
# index 2 = Healthy
HEALTHY_CLASS_ID = 2

def get_gt_fracture(label_path: Path) -> int:
    """
    Return 1 if fractured, 0 if healthy.
    Healthy = all annotations are class 2.
    Fractured = at least one annotation != 2.
    If no label file or empty -> treat as healthy (0).
    """
    if not label_path.exists():
        return 0  # no label -> healthy
    text = label_path.read_text().strip()
    if not text:
        return 0  # empty -> healthy
    lines = text.splitlines()
    is_fracture = False
    for line in lines:
        parts = line.split()
        if not parts:
            continue
        cls_id = int(float(parts[0]))
        if cls_id != HEALTHY_CLASS_ID:
            is_fracture = True
            break
    return 1 if is_fracture else 0

def evaluate_split(split_name: str, img_dir: Path, lbl_dir: Path, conf: float = 0.10):
    img_paths = [p for p in img_dir.glob("*.*") if p.suffix.lower() in [".jpg", ".jpeg", ".png"]]
    tp = tn = fp = fn = 0

    for img_path in img_paths:
        lbl_path = lbl_dir / f"{img_path.stem}.txt"
        gt = get_gt_fracture(lbl_path)

        preds = model.predict(source=str(img_path), imgsz=640, conf=conf, verbose=False)
        dets = preds[0].boxes
        pred = 1 if dets is not None and len(dets) > 0 else 0

        if gt == 1 and pred == 1:
            tp += 1
        elif gt == 0 and pred == 0:
            tn += 1
        elif gt == 0 and pred == 1:
            fp += 1
        else:  # gt == 1 and pred == 0
            fn += 1

    total = tp + tn + fp + fn
    acc = (tp + tn) / total if total else 0
    prec = tp / (tp + fp) if (tp + fp) else 0
    rec = tp / (tp + fn) if (tp + fn) else 0
    f1 = (2 * prec * rec / (prec + rec)) if (prec + rec) else 0

    return {
        "split": split_name,
        "total": total,
        "tp": tp, "tn": tn, "fp": fp, "fn": fn,
        "accuracy": acc,
        "precision": prec,
        "recall": rec,
        "f1": f1,
    }

all_results = []
for split, img_dir in SPLITS.items():
    lbl_dir = LABEL_BASE[split]
    res = evaluate_split(split, img_dir, lbl_dir, conf=0.10)
    all_results.append(res)

# print nicely
for r in all_results:
    print(f"\n=== {r['split'].upper()} ===")
    print(f"Total: {r['total']}")
    print(f"TP: {r['tp']}  TN: {r['tn']}  FP: {r['fp']}  FN: {r['fn']}")
    print(f"Accuracy:  {r['accuracy']:.4f}")
    print(f"Precision: {r['precision']:.4f}")
    print(f"Recall:    {r['recall']:.4f}")
    print(f"F1-score:  {r['f1']:.4f}")

# overall (optional): combine all
overall = {
    "tp": sum(r["tp"] for r in all_results),
    "tn": sum(r["tn"] for r in all_results),
    "fp": sum(r["fp"] for r in all_results),
    "fn": sum(r["fn"] for r in all_results),
}
total = overall["tp"] + overall["tn"] + overall["fp"] + overall["fn"]
overall_acc = (overall["tp"] + overall["tn"]) / total if total else 0
overall_prec = overall["tp"] / (overall["tp"] + overall["fp"]) if (overall["tp"] + overall["fp"]) else 0
overall_rec = overall["tp"] / (overall["tp"] + overall["fn"]) if (overall["tp"] + overall["fn"]) else 0
overall_f1 = (2 * overall_prec * overall_rec / (overall_prec + overall_rec)) if (overall_prec + overall_rec) else 0

print("\n=== OVERALL (train+val+test) ===")
print(f"TP: {overall['tp']}  TN: {overall['tn']}  FP: {overall['fp']}  FN: {overall['fn']}")
print(f"Accuracy:  {overall_acc:.4f}")
print(f"Precision: {overall_prec:.4f}")
print(f"Recall:    {overall_rec:.4f}")
print(f"F1-score:  {overall_f1:.4f}")



=== TRAIN ===
Total: 5536
TP: 1221  TN: 623  FP: 30  FN: 3662
Accuracy:  0.3331
Precision: 0.9760
Recall:    0.2501
F1-score:  0.3981

=== VAL ===
Total: 128
TP: 42  TN: 7  FP: 0  FN: 79
Accuracy:  0.3828
Precision: 1.0000
Recall:    0.3471
F1-score:  0.5153

=== TEST ===
Total: 64
TP: 13  TN: 3  FP: 0  FN: 48
Accuracy:  0.2500
Precision: 1.0000
Recall:    0.2131
F1-score:  0.3514

=== OVERALL (train+val+test) ===
TP: 1276  TN: 633  FP: 30  FN: 3789
Accuracy:  0.3333
Precision: 0.9770
Recall:    0.2519
F1-score:  0.4006
